In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [27]:
x_train, y_train,x_test_original, y_test_original = ReadTS('OSUleaf')
y_train = y_train - 1
y_test_original = y_test_original - 1
#y_train = np.where(y_train == -1, 0, y_train)
#y_test_original = np.where(y_test_original == -1, 0, y_test_original)
print(x_train.shape)
print(y_train)

(200, 427, 1)
[5. 4. 1. 0. 2. 2. 1. 2. 3. 4. 0. 1. 2. 4. 1. 1. 1. 3. 3. 0. 4. 3. 3. 3.
 0. 5. 2. 4. 0. 3. 3. 5. 4. 4. 1. 4. 3. 0. 3. 1. 4. 4. 3. 4. 4. 2. 0. 2.
 1. 3. 2. 0. 2. 4. 3. 1. 2. 3. 5. 3. 0. 3. 3. 0. 3. 2. 3. 0. 1. 0. 3. 4.
 4. 0. 0. 3. 4. 3. 3. 3. 3. 3. 1. 2. 1. 2. 0. 0. 3. 5. 4. 3. 0. 2. 1. 1.
 4. 3. 2. 1. 4. 3. 0. 3. 3. 2. 3. 4. 3. 2. 5. 3. 3. 0. 2. 2. 4. 4. 3. 0.
 3. 5. 5. 0. 1. 2. 0. 4. 4. 3. 3. 4. 0. 2. 2. 1. 0. 4. 3. 2. 2. 2. 4. 3.
 1. 3. 3. 3. 1. 3. 2. 5. 4. 1. 1. 5. 5. 3. 0. 1. 1. 2. 0. 4. 2. 2. 4. 4.
 3. 5. 3. 4. 0. 0. 4. 3. 0. 5. 1. 1. 3. 5. 3. 3. 4. 2. 0. 1. 0. 0. 4. 1.
 2. 4. 5. 0. 2. 0. 1. 2.]


In [28]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    # 获取原始数据集的形状
    original_shape = TS.shape
    
    # 将数据集reshape成二维数组
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    # 初始化StandardScaler
    scaler = StandardScaler()
    
    # 对数据集进行标准化
    TS_normalized = scaler.fit_transform(TS_2d)
    
    # 将标准化后的数据集reshape回原始形状
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [35]:
model = make_CNN_model(x_train.shape[1:],num_classes = 6)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=500, batch_size=32)

Epoch 1/500
5/5 [==============================] - 0s 41ms/step - loss: 1.8050 - sparse_categorical_accuracy: 0.1562 - val_loss: 1.7903 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/500
5/5 [==============================] - 0s 23ms/step - loss: 1.6204 - sparse_categorical_accuracy: 0.2688 - val_loss: 1.7920 - val_sparse_categorical_accuracy: 0.1500
Epoch 3/500
5/5 [==============================] - 0s 22ms/step - loss: 1.5181 - sparse_categorical_accuracy: 0.3688 - val_loss: 1.7934 - val_sparse_categorical_accuracy: 0.1500
Epoch 4/500
5/5 [==============================] - 0s 22ms/step - loss: 1.4327 - sparse_categorical_accuracy: 0.4625 - val_loss: 1.7954 - val_sparse_categorical_accuracy: 0.1500
Epoch 5/500
5/5 [==============================] - 0s 24ms/step - loss: 1.3802 - sparse_categorical_accuracy: 0.4812 - val_loss: 1.7988 - val_sparse_categorical_accuracy: 0.1500
Epoch 6/500
5/5 [==============================] - 0s 21ms/step - loss: 1.3445 - sparse_categorical_accuracy: 

In [36]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

8/8 [==============================] - 0s 4ms/step - loss: 1.0133 - sparse_categorical_accuracy: 0.6529
Test accuracy 0.6528925895690918
Test loss 1.013289451599121


### Create perturbed test sets and calculate decrease in accuracy

In [37]:
print(y_test_original.shape)

(242,)


In [119]:
predictions_original = model.predict(x_test_original)
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 320
index_1 = 400
x_test_rbp = np.asarray([perturb('RBP',x,index_0,index_1) for x in perturb_test_set])
x_test_rbp = np.expand_dims(x_test_rbp, axis=-1)
predictions_rbp = model.predict(x_test_rbp)

bg = multisamplebackgroundIdentification(perturb_test_set)
x_test_new1 = np.asarray([perturb('RBP1fast',x,index_0,index_1, bg = bg) for x in perturb_test_set])
x_test_new1 = np.expand_dims(x_test_new1, axis=-1)
predictions_new1 = model.predict(x_test_new1)

#x_test_new2 = np.asarray([perturb('RBP2',x,400,500, global_ts = perturb_test_set) for x in perturb_test_set])
#x_test_new2 = np.expand_dims(x_test_new2, axis=-1)
#predictions_new2 = model.predict(x_test_new2)

x_test_zero = np.asarray([perturb('zero',x,index_0,index_1) for x in perturb_test_set])
x_test_zero = np.expand_dims(x_test_zero, axis=-1)
predictions_zero = model.predict(x_test_zero)

x_test_noise = np.asarray([perturb('noise',x,index_0,index_1) for x in perturb_test_set])
x_test_noise = np.expand_dims(x_test_noise, axis=-1)
predictions_noise = model.predict(x_test_noise)

x_test_blur = np.asarray([perturb('blur',x,index_0,index_1) for x in perturb_test_set])
x_test_blur = np.expand_dims(x_test_blur, axis=-1)
predictions_blur = model.predict(x_test_blur)

8/8 [==============================] - 0s 4ms/step


In [120]:
def l2norm(perturb):
    # 计算两个数组之间的差
    difference = perturb - x_test_original
    
    # 计算差的平方
    squared_difference = np.square(difference)
    
    # 计算平方和，沿着特征维度（axis=1），因为每个时间序列样本是一行
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    # 计算平方和的平方根，即l2范数
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    # l2_norms现在是一个形状为(467,)的数组，其中每个元素代表
    # 对应时间序列样本之间的l2范数
    print(average_l2_norm)  # 输出: (467,)

l2norm(x_test_rbp)
l2norm(x_test_new1)
l2norm(x_test_zero)
l2norm(x_test_noise)
l2norm(x_test_blur)

7.637847481850354
8.651312168427248
8.614009877871524
14.43180193903959
0.16182532004972403


In [121]:
def accuracy(predictions, labels):
    hits = 0
    for i in range(0,len(predictions)):
        if predictions[i] == labels[i]:
            hits += 1
    return hits/len(predictions)

In [122]:
original_accuracy = accuracy(np.argmax(predictions_original,axis=1), y_test_original)
rbp_accuracy = accuracy(np.argmax(predictions_rbp,axis=1), y_test_original)
new1_accuracy = accuracy(np.argmax(predictions_new1,axis=1), y_test_original)
#new2_accuracy = accuracy(np.argmax(predictions_new2,axis=1), y_test_original)
zero_accuracy = accuracy(np.argmax(predictions_zero,axis=1), y_test_original)
noise_accuracy = accuracy(np.argmax(predictions_noise,axis=1), y_test_original)
blur_accuracy = accuracy(np.argmax(predictions_blur,axis=1), y_test_original)

In [123]:
print(" Original Accuracy: " + str(original_accuracy) + "\n"+
    " RBP Accuracy: " + str(rbp_accuracy) + "\n"+
    " RBP1 Accuracy: " + str(new1_accuracy) + "\n"+
    #" RBP2 Accuracy: " + str(new2_accuracy) + "\n"+
      
    " Zero Accuracy: " + str(zero_accuracy) + "\n"+
    " Noise Accuracy: " + str(noise_accuracy) + "\n"+
    " Blur Accuracy: " + str(blur_accuracy) + "\n")

 Original Accuracy: 0.6528925619834711
 RBP Accuracy: 0.5082644628099173
 RBP1 Accuracy: 0.2066115702479339
 Zero Accuracy: 0.3884297520661157
 Noise Accuracy: 0.1322314049586777
 Blur Accuracy: 0.6942148760330579

